In [1]:
from sklearn import metrics 
import tensorflow as tf
import torch
import numpy as np
import pickle
import time
import os
torch.set_printoptions(precision=8)

In [5]:
repeat = 341
use_gpu = True
num_articles=100
num_arm_features=1536
alpha=0.1
query_feature_num=768
page_feature_num=768


In [2]:
query_features = pickle.load(open('./msmarco/X_question_bert.pkl','rb'))
page_features=pickle.load(open('./msmarco/X_page_bert.pkl','rb'))
    
test_query_features= pickle.load(open('./msmarco/X_question_bert.pkl','rb'))

ratings = pickle.load(open('./msmarco/X_ass_mat.pkl','rb'))
test_ratings = pickle.load(open('./msmarco/X_ass_mat.pkl','rb'))

K_pc_500 = pickle.load(open('./msmarco/K_arm_bert_raw.pkl','rb'))

In [3]:
X_nystrom = tf.constant(K_pc_500, dtype=tf.float32, name="X_nystrom")

In [7]:
qt_id = tf.constant(0)
qt_id_to_arm_ids = [[armloop*num_articles+qt_id] for armloop in range(num_articles)] 
k=tf.gather_nd(X_nystrom, tf.convert_to_tensor(qt_id_to_arm_ids))

In [10]:
len(qt_id_to_arm_ids)

100

In [11]:
k.shape

TensorShape([100, 1536])

In [16]:
qt_id = 0
loc = torch.ByteTensor([armloop*num_articles+qt_id for armloop in range(num_articles)])
X_nystrom = torch.tensor(K_pc_500, dtype=torch.float32)

In [23]:
loc = [armloop*num_articles+qt_id for armloop in range(num_articles)]

In [25]:
X_nystrom[loc]

tensor([[ 0.24776939,  0.14772353, -0.15037628,  ..., -0.12566939,
          0.12129939,  0.24182221],
        [ 0.41537824,  0.55095118, -0.28244665,  ..., -0.12566939,
          0.12129939,  0.24182221],
        [ 0.27643803,  0.03907634,  0.05725826,  ..., -0.12566939,
          0.12129939,  0.24182221],
        ...,
        [ 0.56506455,  0.08213788, -0.17778534,  ..., -0.12566939,
          0.12129939,  0.24182221],
        [ 0.28807366, -0.00948920,  0.05025373,  ..., -0.12566939,
          0.12129939,  0.24182221],
        [-0.14007340, -0.35157716, -0.09985719,  ..., -0.12566939,
          0.12129939,  0.24182221]])

In [26]:
k

<tf.Tensor: id=607, shape=(100, 1536), dtype=float32, numpy=
array([[ 0.24776939,  0.14772353, -0.15037628, ..., -0.12566939,
         0.12129939,  0.24182221],
       [ 0.41537824,  0.5509512 , -0.28244665, ..., -0.12566939,
         0.12129939,  0.24182221],
       [ 0.27643803,  0.03907634,  0.05725826, ..., -0.12566939,
         0.12129939,  0.24182221],
       ...,
       [ 0.56506455,  0.08213788, -0.17778534, ..., -0.12566939,
         0.12129939,  0.24182221],
       [ 0.28807366, -0.0094892 ,  0.05025373, ..., -0.12566939,
         0.12129939,  0.24182221],
       [-0.1400734 , -0.35157716, -0.09985719, ..., -0.12566939,
         0.12129939,  0.24182221]], dtype=float32)>

In [28]:
x = torch.tensor([1, 2, 3, 4])
torch.unsqueeze(x, -1)

tensor([[1],
        [2],
        [3],
        [4]])

In [32]:
# x = tf.constant([1,2,3,4])
tf.expand_dims(k, -1).shape

TensorShape([100, 1536, 1])

In [26]:
def compute_cosine_similarity_tf(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b
    #print('a ', a)
    normalize_a = tf.math.l2_normalize(a,0)        
    #print('normalize_a ', normalize_a)

    #print('b ', b)
    normalize_b = tf.math.l2_normalize(b,0)
    #print('normalize_b ', normalize_b)

    similarity = tf.tensordot(normalize_a, normalize_b, (0,0))
    #print('returning ', similarity)
    return similarity

In [37]:
import torch.nn.functional as f
def compute_cosine_similarity_pt(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim

    normalize_a = f.normalize(a, dim=0, p=2)
    normalize_b = f.normalize(b, dim=0, p=2)

    similarity = torch.tensordot(normalize_a, normalize_b, ([0], [0]))

    return similarity

In [28]:
a=np.random.rand(768, 49)
b=np.random.rand(768, 1)

In [38]:
sim_pt = compute_cosine_similarity_pt(torch.tensor(a), torch.tensor(b))

In [39]:
sim_tf = compute_cosine_similarity_tf(tf.constant(a), tf.constant(b))

In [42]:
sim_pt.shape

torch.Size([49, 1])

In [44]:
sim_tf

<tf.Tensor: id=62, shape=(49, 1), dtype=float64, numpy=
array([[0.75272832],
       [0.75002452],
       [0.74601698],
       [0.73950791],
       [0.74133337],
       [0.77542177],
       [0.74688002],
       [0.73135711],
       [0.74138545],
       [0.74913381],
       [0.76236682],
       [0.76214455],
       [0.75092048],
       [0.74854653],
       [0.75212117],
       [0.72937608],
       [0.7408579 ],
       [0.75335998],
       [0.76462885],
       [0.73569345],
       [0.74048226],
       [0.74269353],
       [0.74967229],
       [0.74230611],
       [0.73540302],
       [0.75240642],
       [0.75008834],
       [0.74507091],
       [0.75776866],
       [0.76347359],
       [0.74089768],
       [0.74235728],
       [0.76729421],
       [0.75115985],
       [0.75417725],
       [0.76085575],
       [0.7512806 ],
       [0.76727526],
       [0.75669488],
       [0.74900206],
       [0.75139084],
       [0.74415422],
       [0.75148004],
       [0.75220564],
       [0.74380281],

In [43]:
sim_pt

tensor([[0.75272832],
        [0.75002452],
        [0.74601698],
        [0.73950791],
        [0.74133337],
        [0.77542177],
        [0.74688002],
        [0.73135711],
        [0.74138545],
        [0.74913381],
        [0.76236682],
        [0.76214455],
        [0.75092048],
        [0.74854653],
        [0.75212117],
        [0.72937608],
        [0.74085790],
        [0.75335998],
        [0.76462885],
        [0.73569345],
        [0.74048226],
        [0.74269353],
        [0.74967229],
        [0.74230611],
        [0.73540302],
        [0.75240642],
        [0.75008834],
        [0.74507091],
        [0.75776866],
        [0.76347359],
        [0.74089768],
        [0.74235728],
        [0.76729421],
        [0.75115985],
        [0.75417725],
        [0.76085575],
        [0.75128060],
        [0.76727526],
        [0.75669488],
        [0.74900206],
        [0.75139084],
        [0.74415422],
        [0.75148004],
        [0.75220564],
        [0.74380281],
        [0

In [36]:
tf.constant(a).shape

TensorShape([768, 49])

In [45]:
x = tf.reshape(tf.constant([[1,2], [3,4], [5,6]]), [6,1,1])
y = tf.reshape(tf.constant([[5,2], [1,4], [2,0]]), [6,1,1])
# v_t = tf.argmax(tf.math.reduce_max(x, axis=0), axis=1)
# tf.argmax(tf.math.reduce_max(x, axis=0),1)

In [46]:
z = tf.concat([x, y], -1)
v_t = tf.argmax(tf.math.reduce_max(z, axis=0), axis=1)
h_t= tf.argmax(z, axis=0) 
a_t = tf.expand_dims(tf.expand_dims(tf.gather_nd(h_t,[0, v_t[0].numpy()]),-1),-1)


In [51]:
z.shape

TensorShape([6, 1, 2])

In [52]:
tf.argmax(z, axis=0) .shape

TensorShape([1, 2])

In [39]:
v_t

<tf.Tensor: id=59, shape=(1,), dtype=int64, numpy=array([0])>

In [40]:
h_t

<tf.Tensor: id=61, shape=(1, 2), dtype=int64, numpy=array([[5, 0]])>

In [41]:
a_t

<tf.Tensor: id=71, shape=(1, 1), dtype=int64, numpy=array([[5]])>

In [31]:
tf.math.reduce_max(z, axis=0)

<tf.Tensor: id=45, shape=(1, 2), dtype=int32, numpy=array([[6, 5]], dtype=int32)>

In [20]:
a = torch.reshape(torch.tensor([[1,2], [3,4], [5,6]]), (6,1,1))
b = torch.reshape(torch.tensor([[5,2], [1,4], [2,0]]), (6,1,1))
# v_t = tf.argmax(tf.math.reduce_max(x, axis=0), axis=1)
# tf.argmax(tf.math.reduce_max(x, axis=0),1)

In [28]:
c = torch.stack([a, b], 2).reshape(-1,1,2)

In [33]:
torch.max(c, dim=0).values

tensor([[6, 5]])

In [34]:
torch.argmax(torch.max(c, dim=0).values, dim=1)

tensor([0])